In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import numpy as np
import matplotlib as mpl


# Fonts for plots
plt.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif'] = 'Times New Roman'

pd.set_option('display.max_columns', None)

# Path to data retrieval and storage
path = "C:/Users/kleinow/ownCloud/MA_Neuro"

In [2]:
# load reference data
cn_refs = pd.read_csv(path + '/cn_refs.csv')
cn_refs.head()
cn_refs.shape # (81694, 16)

C:\Users\kleinow\AppData\Local\Temp\ipykernel_20828\2191522443.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  cn_refs = pd.read_csv(path + '/cn_refs.csv')


(81694, 16)

Removal of any full duplicate rows

In [3]:
cn_refs = cn_refs.drop_duplicates().reset_index(drop=True) # considers the entire row duplicates
cn_refs.shape # (81694, 16); no full duplicate rows in the retrieved data set, which is reasonable

(81694, 16)

Adding a column that contains the cited titles in lower-case

In [8]:
cn_refs['title_lower'] = cn_refs['ref_item_title'].str.lower()
#cn_refs.head()

cn_refs.title_lower.nunique() # 49802, drops na as default

49802

Trying out the first citation threshold before final cleaning: cited more than once, otherwise no connection to other papers possible

In [12]:
series_cn_refs = cn_refs.groupby("item_id_cited")['item_id_citing'].nunique() #  Group the cited id by the citing id
#print(series_cn_refs)

# Cited items with more than one citing item
ab1_cn_refs = series_cn_refs.loc[series_cn_refs.values > 1]

# Show positions of the cited items with more than one citing item
#cn_refs.loc[cn_refs.item_id_cited.isin(ab1_cn_refs.index)] # 30914 rows × 17 columns

,item_id_citing,item_id_cited,citing_pubyear,ref_seq_nr,ref_pubyear,ref_item_title,ref_source_title,ref_authors,ref_volume,scopus_ref_issue,ref_pages,ref_doi,wos_ref_article_number,scopus_ref_text,scopus_ref_fulltext,wos_citation_context,title_lower
4,WOS:000086279300008,WOS:A1995RP75600044,2000,5,1995.0,MODULATION OF MEMORY FIELDS BY DOPAMINE D1 REC...,NATURE,"{""WILLIAMS, GV""}",376,NaN,572,NaN,NaN,NaN,NaN,NaN,modulation of memory fields by dopamine d1 rec...
6,WOS:000086279300008,WOS:A1996VV46700007,2000,7,1996.0,Regional and cellular fractionation of working...,PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCE...,"{""GoldmanRakic, PS""}",93,NaN,13473,NaN,NaN,NaN,NaN,NaN,regional and cellular fractionation of working...
13,WOS:000086279300008,WOS:A1985ARE2300019,2000,14,1985.0,PRIMATE FRONTAL EYE FIELDS .2. PHYSIOLOGICAL A...,JOURNAL OF NEUROPHYSIOLOGY,"{""BRUCE, CJ""}",54,NaN,714,NaN,NaN,NaN,NaN,NaN,primate frontal eye fields .2. physiological a...
14,WOS:000086279300008,WOS:A1980JC97100014,2000,15,1980.0,DISSOCIATION OF VISUAL AND SACCADE-RELATED RES...,JOURNAL OF NEUROPHYSIOLOGY,"{""MAYS, LE""}",43,NaN,207,NaN,NaN,NaN,NaN,NaN,dissociation of visual and saccade-related res...
15,WOS:000086279300008,WOS:A1985ADU5200001,2000,16,1985.0,PRIMATE FRONTAL EYE FIELDS .1. SINGLE NEURONS ...,JOURNAL OF NEUROPHYSIOLOGY,"{""BRUCE, CJ""}",53,NaN,603,NaN,NaN,NaN,NaN,NaN,primate frontal eye fields .1. single neurons ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81683,WOS:A1995RP56000003,WOS:A1982PK72800012,1995,118,1982.0,RECEPTIVE-FIELD PROPERTIES OF X-CELLS AND Y-CE...,VISION RESEARCH,"{""LINSENMEIER, RA""}",22,NaN,1173,NaN,NaN,NaN,NaN,NaN,receptive-field properties of x-cells and y-ce...
81684,WOS:A1995RP56000003,BCI:BCI199497521809,1995,119,1994.0,Linearity of synaptic interactions in the asse...,Current Opinion in Neurobiology,"{""Ferster, David""}",4,NaN,563,NaN,NaN,NaN,NaN,NaN,linearity of synaptic interactions in the asse...
81685,WOS:A1995RP56000003,WOS:A19636746B00014,1995,120,1963.0,SHAPE AND ARRANGEMENT OF COLUMNS IN CATS STRIA...,JOURNAL OF PHYSIOLOGY-LONDON,"{""HUBEL, DH""}",165,NaN,559,NaN,NaN,NaN,NaN,NaN,shape and arrangement of columns in cats stria...
81688,WOS:A1995RP56000003,WOS:A1991ER41700006,1995,123,1991.0,SYNAPTIC TRANSMISSION BETWEEN INDIVIDUAL PYRAM...,JOURNAL OF NEUROSCIENCE,"{""MASON, A""}",11,NaN,72,NaN,NaN,NaN,NaN,NaN,synaptic transmission between individual pyram...


Creating dictionaries to rename duplicates of cited reference ids that accidentally were assigned a new ID; adding a new column with the revised ids

In [13]:

dictcn={}

for index,s in cn_refs.iterrows(): # Loop over rows in our df
    if s['title_lower'] in dictcn: # If title already exists in our created dictionary,
        if s['item_id_cited'] not in dictcn[s['title_lower']]: # and id_item_cited is still not present in our dictionary...
            dictcn[s['title_lower']].append(s['item_id_cited']) # then append the id_item_cited to the existing title
    else: 
        dictcn[s['title_lower']] = [s['item_id_cited']] #if not, create new key title with item_id_cited as a value
        
len(dictcn) # 49803

49803

In [16]:
# Check if the dictionary is correct

print(list(dictcn.items())[:100])

chunk_dictcn = {k: dictcn[k] for k in list(dictcn)[:20]}
print(chunk_dictcn) # Gives chunk of 20 titles

[('cortex of cerebellum', ['WOS:A1975V163400004']), (nan, ['WOS:000086279300008.22', 'WOS:000086279300008.30', 'WOS:000086279300008.13', 'WOS:000086279300008.1', 'WOS:000086279300008.29', 'WOS:000086279300008.8', 'WOS:000086279300008.10', 'WOS:000086279300008.21', 'WOS:000086279300008.4', 'WOS:000086279300008.9', 'WOS:000086279300008.24', 'WOS:000086279300008.11', 'WOS:000086279300008.2', 'WOS:000086279300008.23', 'WOS:000088763200026.15', 'WOS:000088763200026.16', 'WOS:000088763200026.21', 'WOS:000088763200026.8', 'WOS:000088763200026.1', 'WOS:000088763200026.20', 'WOS:000088763200026.19', 'WOS:000088763200026.2', 'WOS:000169285300025.20', 'WOS:000169285300025.25', 'WOS:000169285300025.13', 'WOS:000169285300025.10', 'WOS:000169285300025.34', 'WOS:000169285300025.31', 'WOS:000169285300025.26', 'WOS:000169285300025.2', 'WOS:000173024600148.9', 'WOS:000173024600148.1', 'WOS:000173024600148.5', 'WOS:000174600200006.1', 'WOS:000174600200006.9', 'WOS:000174600200006.4', 'WOS:000174600200006

Dictionary Key-Value pairs with more than 1 entry need to be relabeled, otherwise the same references might have different labels.

In [17]:
dictdupcn = {k: dictcn[k] for k in dictcn if len(dictcn[k]) > 1} 

#print(dictdupcn)
len(dictdupcn) # 549

#cn_refs.isna()
#print(sorted(dictdupcn.items(), key= lambda x: len(x[1]), reverse=True) ) # Sort by number of values

del dictdupcn[np.nan] # nan needs the np in front!!
dictdupcn_sorted = sorted(dictdupcn.items(), key= lambda x: len(x[1]), reverse=True) # Sort by number of values

In [18]:
print(dictdupcn_sorted)

[('nest (neural simulation tool)', ['000311837300002.31', '000345024600003.9', '000579856100001.12', '000348207000001.18', '000370606600001.25', '000426546200001.10', '000430129400001.34', '000455034000009.6', '000596843300001.10', '000642586500006.34', '000646233500012.35', '000647460100001.11', '000876518600001.2', '000934089300002.12', '000209207100026.14', '000305415000005.10', '000373641800003.28', '000425314200001.19', '000444235500012.19', '000804814900001.23', '000311837300008.10', '000343228000001.19', '000419789500012.11', '000429596100002.15', '000499854500001.13', '000806558700001.11', '000384452300003.11', '000432839000001.10', '000460392200012.15', '000656962800012.3', '000804029700001.21', '000810997200001.31', '000386313100026.2', '000453105700001.32', '000450516000020.58', '000502758100001.16', '000528677400001.33', '000209207300033.3', '000305415000003.53', '000311837300007.19', '000348109300001.11', '000380668600001.14', '000478905500001.22', '000726266400001.17', '0

Create dataframe for lower-case titles and the assigned multiple IDs

In [22]:
# create df for cn_refs multiple ids per lower case reference title
dfdupcn=pd.DataFrame.from_dict(dict(dictdupcn_sorted), orient='index') # df for better visibility
dfdupcn.to_csv('cn_refs_titles_with_multi_ids.csv', encoding='utf-8')
dfdupcn.head(20) # 796 rows × 63 columns

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62
nest (neural simulation tool),000311837300002.31,000345024600003.9,000579856100001.12,000348207000001.18,000370606600001.25,000426546200001.10,000430129400001.34,000455034000009.6,000596843300001.10,000642586500006.34,000646233500012.35,000647460100001.11,000876518600001.2,000934089300002.12,000209207100026.14,000305415000005.10,000373641800003.28,000425314200001.19,000444235500012.19,000804814900001.23,000311837300008.10,000343228000001.19,000419789500012.11,000429596100002.15,000499854500001.13,000806558700001.11,000384452300003.11,000432839000001.10,000460392200012.15,000656962800012.3,000804029700001.21,000810997200001.31,000386313100026.2,000453105700001.32,000450516000020.58,000502758100001.16,000528677400001.33,000209207300033.3,000305415000003.53,000311837300007.19,000348109300001.11,000380668600001.14,000478905500001.22,000726266400001.17,000736022600001.28,000805555900001.27,000337948500002.17,000348206200001.20,000454180100015.9,000495242400010.13,000811824200001.16,000937441800001.35,000963727400001.27,000209207300041.9,000380500900617.4,000396398700015.18,000406561700001.20,000449250100001.27,000536333100001.13,000581985500047.50,000823396700001.27,000859912400038.12,000933363400001.62
imagenet classification with deep convolutional neural networks,000395099500043.47,000432199000012.32,000577089300003.38,000626471200001.37,INSPEC:17133663,WOS:000402555400026,000443157700005.85,000486630500017.96,000744537200017.8,000434779300020.36,000472127600009.32,000595874700013.1,000922928209014.34,000379319900018.56,000495400000052.72,000621797000017.26,000518892900013.40,000390601400001.13,000575874100001.47,000336976000076.20,000457636800259.19,000383808700002.32,000429191800009.92,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
very deep convolutional networks for large-scale image recognition,000577089300003.56,000739027800001.63,000744537200017.14,000370822200004.62,000472127600009.74,000595874700013.2,000883330900010.19,000621797000017.37,000416196400023.20,000442861600066.26,000617031000014.154,000776460100001.53,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
adam: a method for stochastic optimization,000548564900074.23,000579856100001.22,000450651000031.8,000577089300003.35,000626471200001.35,000870207000002.14,000555729900082.11,000617614000002.60,000612948000029.6,000922928203018.23,000465513800001.27,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
learning from the past: approaches for reproducibility in computational neuroscience,000311837300002.17,000429167300001.37,000384570200006.37,000430129400001.18,000454422200001.15,000431227400001.28,000348109300001.9,000209207300041.3,000933363400001.49,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
a logical calculus of the ideas immanent in nervous activity,BCI:BCI19441800008456,000597437600001.101,000733527400001.1,000335628800008.30,000379319900018.66,000432567500008.267,000443568600010.43,000366638300005.40,000429191800009.105,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None

The titles with multiple references need to be checked manually in one of the following steps. For very specific titles and similar meta information, we can assume the publications to actually be the same.

Replacing dictionary and adding new column with clean item_id_cited

In [23]:
cn_refs['item_id_clear'] = cn_refs['item_id_cited'] # Clone item_id_cited column to later replace values only in there
cn_refs.head() # new column created
type(dictdupcn) # dict

dictreplace_cn ={v[0]: v[1:] for v in dictdupcn.values()} 
# ignore keys aka titles; take first id as key, remaining ids until last as values
# goal: {'BCI:BCI19522600023673': ["A", "B"]} 
dfdictreplace_cn=pd.DataFrame.from_dict(dict(dictdupcn_sorted), orient='index') # df for better visibility
dfdictreplace_cn
# rearranging dictionary to long format to avoid double looping through df and value lists

dictreplace_cn_alt = {i: k for k,v in dictreplace_cn.items() for i in v} # needs .items(), otherwise returns meta stuff
# k becomes first id
# v is remaining list of other ids

# same as:
#dictreplace_alt = {}
#for k,v in dictreplace.items():
#    for i in v:
#        dictreplace_alt[i]=k

dictreplace_cn_alt

{'000447832000018.64': 'WOS:000083883300021',
 '000553719400001.16': 'WOS:A1997BH93C00030',
 '000452649401032.23': 'WOS:A1952UH81500008',
 '000402008600074.5': 'WOS:A1952UH81500008',
 '000301568100007.23': 'WOS:000171017300008',
 '000331486100014.7': 'WOS:000167248500017',
 'WOS:A1990BT68K00088': 'WOS:A1990HB91800089',
 '000313591500023.3': 'WOS:000231235700016',
 '000460855900002.174': '000209165300094.87',
 '000486417800002.95': '000209165300094.87',
 'WOS:A1983RE67600006': 'WOS:000275149200009',
 '000298103700001.5': 'WOS:000208047200002',
 '000339052800001.22': '000209204100004.15',
 '000345024600005.34': '000209529000006.10',
 'WOS:000230925500136': '000209529000006.14',
 '000426982100013.18': '000209529000006.14',
 'WOS:000082395600005': 'WOS:000221563000002',
 '000397326500005.11': 'WOS:A1997XM55000001',
 '000373939100076.8': 'WOS:A1997XM55000001',
 '000337948500002.24': 'WOS:A1997XM55000001',
 '000312652100120.5': 'CCC:000079284100002',
 'WOS:A1945UH01400003': 'WOS:000200527300

In [24]:
cn_refs2 = cn_refs.replace({'item_id_clear':dictreplace_cn_alt})
cn_refs.compare(cn_refs2) #1337 rows × 2 columns

item_id_clear                     
                      self                other
576    WOS:A1945UH01400003  WOS:000200527300004
1885    000301568100007.23  WOS:000171017300008
2134   WOS:000283727800004    000299100900007.2
2568   WOS:000176839200031   000333946600016.52
2974     000345024600003.9   000311837300002.31
...                    ...                  ...
81337   000967419400001.67  WOS:000389086300036
81339   000967419400001.13   000459740300017.32
81457  WOS:A1982NV80900008  WOS:A1982PS83700005
81483  BCI:BCI197152109060    000316056900014.2
81530  WOS:A1990BT68K00088  WOS:A1990HB91800089

[1337 rows x 2 columns]

Now, the item_ids of the references, that have multiple IDs for the same tile, are checked manually

In [29]:
pd.set_option('display.max_rows', None) # Show all rows

In [31]:
# check titles with most ids, take care of indexing
# cn_refs2.loc[cn_refs2['title_lower'] == 'nest (neural simulation tool)'] # uncomment to see all rows
# Nest (neural simulation tool)	refers to a computer program by Gewaltig and Diesmann. All references refer to the same.

In [34]:
#cn_refs2.loc[cn_refs2['title_lower'] == 'imagenet classification with deep convolutional neural networks']
# All the same reference, Weinberger refers to Eds., In: Pereira, F., Burges, C.J.C., Bottou, L. and Weinberger, K.Q., Eds., Advances in Neural Information Processing Systems, Vol. 25, Curran Associates, Inc., Red Hook, NY, 1097-1105.